In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint, shuffle
import sys

In [2]:
sys.version

'3.7.5 (default, Oct 31 2019, 15:18:51) [MSC v.1916 64 bit (AMD64)]'

## Import Datasets

In [4]:
data_x = pd.read_csv('dataset_X.csv').iloc[:,1:]
data_t = pd.read_csv('dataset_T.csv').iloc[:,1:]

## Solve Equation

### define M for poly

In [5]:
def two(x):
    sq = pd.DataFrame()
    for i in range(x.shape[1]):
        for j in range(x.shape[1]):
            if j >= i:
                row = []
                for k in range(x.shape[0]):
                    row.extend([x.iloc[k, i]*x.iloc[k, j]])
                str = data_x.columns.values[i]+'*'+data_x.columns.values[j]
                sq[str] = row
    return sq

### define $\Phi$ matrix

$$\phi_j(x_i)= \Sigma \ \Pi_{i=0}^j x_i$$

In [6]:
def get_polyphi(x, M):
    inter = pd.DataFrame({'C': np.repeat(1, len(x), axis=0).transpose()}).astype('float32').reset_index(drop=True)
    onep = x.reset_index(drop=True)
    sq = two(x).reset_index(drop=True)
    
    if M == 0:
        phi = pd.concat([inter], axis=1, ignore_index=True).values
    elif M == 1:
        phi = pd.concat([inter, onep], axis=1, ignore_index=True).values
    elif M == 2:
        phi = pd.concat([inter, onep, sq], axis=1, ignore_index=True).values
    else:
        return print("invalid M")
    return phi

$$\phi_j(x)=\rm{exp}\left\{-\frac{(x-\mu_j)^2}{2s^2}\right\},\ where\ s = 0.1$$

In [7]:
def get_gaussphi(x, M):
    # s = np.std(x)
    # s = 0.1
    if M > x.shape[1]:
        return "M input invalid"
    else:
        inter = pd.DataFrame({'C': np.repeat(1, len(x), axis=0).transpose()}).astype('float32').reset_index(drop=True)
        f = lambda x: np.exp(-((x-np.mean(x))**2)/(2*(np.std(x))**2))
        phi = pd.concat([inter, x.apply(f).iloc[:, 0:M]], axis=1, ignore_index=True)
    return phi

### Solve $w$

$$w = (\lambda I + \Phi^T  \Phi)^{-1} \Phi^T  y$$

In [8]:
def solve_reg(x, y, M, lam, f):
    y = y.values
    phi = f(x, M)    
    w = np.linalg.inv(lam* np.eye(phi.shape[1]) + phi.T @ phi) @ phi.T @ y
    return (w, phi)

## define Cross Validation

### split data

In [9]:
def get_ID(dataset):
    list0 = ([x for x in range(dataset.shape[0])])
    shuffle(list0)
    return list0

def split_ID(list0, val_size:float):
    return ([list0[i:i + int(val_size*len(list0))] for i in range(0, len(list0), int(val_size*len(list0)))])

def split_ID2(list0, piles):
    p = [[] for _ in range(piles)]
    for i in range(len(list0)):
        for j in range(piles):
            if i % piles == j:
                p[j].append(list0[i])
    return [p][0]

### CV

In [10]:
def cross_val(x, t, piles, M, lam, fun):
    """
    Parameters
    ------------
    x, t: training and target data
    piles: split data into piles
    M: hyperparameters for basis function
    lam: regularization term
    fun: basis function
    """
    p = split_ID2(get_ID(x), piles)
    CV_RMSE = []
    train_RMSE = []
    for i in range(len(p)):
        cv_x = x.loc[p[i]].reset_index(drop=True)
        cv_t = t.loc[p[i]].reset_index(drop=True)
        sub_x = x.loc[~x.index.isin(p[i])].reset_index(drop=True)
        sub_t = t.loc[~t.index.isin(p[i])].reset_index(drop=True)
        w, phi = solve_reg(sub_x, sub_t, M, lam, fun)
        res_CV = ((np.ones(cv_t.shape[0]) @ ((fun(cv_x, M) @ w - cv_t.values)**2))/cv_t.shape[0])**0.5
        res_train = ((np.ones(sub_t.shape[0]) @ ((phi @ w - sub_t.values)**2))/sub_t.shape[0])**0.5
        CV_RMSE.append(res_CV[0])
        train_RMSE.append(res_train[0])
        if res_CV[0] == min(CV_RMSE):
            w_m = w
            i_L = i
            #print(i, "update")
        RMSE = pd.DataFrame({'train_RMSE': train_RMSE, 'CV_RMSE': CV_RMSE})
    print("by RMSE, we choose fold {} for the training.".format(i_L))
    return RMSE, w_m

## define search function for hyperparameter  

In [11]:
def search_hyper(x, t, piles, M, lam, fun):
    RMSE_min = float('inf')
    i_min = None

    for i in range(M):
        print("iter: {}".format(i))
        M_G = cross_val(x, t, piles, i, lam, fun)
        if np.min(M_G[0].iloc[:, 1:])[0] < RMSE_min:
            RMSE_min = np.min(M_G[0].iloc[:, 1:])[0]
            i_min = i
            w_opt = M_G[1]
    print("\nFrom iteration {}, we got the lowest CV_RMSE: {}, and the weight has save to w_opt".format(i_min, RMSE_min))
    return [w_opt, i_min]

## define MAP process

In [12]:
def get_post(x, y, s0, m0, M):
    beta = 0.2
    y = y.values
    phi = get_gaussphi(x, M)
    sn = np.linalg.inv(np.linalg.inv(s0) + beta*(phi.T @ phi))
    mn = sn @ (np.linalg.inv(s0) @ m0 + beta*(phi.T @ y))
    return [sn, mn]

In [13]:
def batch_learn(x, t, piles, s0, m0, M):
    p = split_ID2(get_ID(x), piles)
    sn = s0
    mn = m0
    for i in range(len(p)):
        sub_x = x.loc[~x.index.isin(p[i])].reset_index(drop=True)
        sub_t = t.loc[~t.index.isin(p[i])].reset_index(drop=True)
        MAP = get_post(sub_x, sub_t, sn, mn, M)
    
    return MAP

## 1. Feature Selection

### a. In the feature selection stage, please apply polynomials of order M = 1 and M = 2 over the dimension D = 17 of input data. **Please evaluate the corresponding RMS error on the training set and validation set.**  

In [14]:
M_2 = cross_val(data_x, data_t, 5, 2, 0, get_polyphi)
M_1 = cross_val(data_x, data_t, 5, 1, 0, get_polyphi)

by RMSE, we choose fold 1 for the training.
by RMSE, we choose fold 4 for the training.


In [15]:
M_1[0]
M_2[0]

,train_RMSE,CV_RMSE
0,4.124690,4.100647
1,3.947910,4.789979
2,4.135423,4.052498
3,4.123886,4.115327
4,4.135633,4.022529


,train_RMSE,CV_RMSE
0,3.272475,4.800359
1,3.330068,4.172624
2,3.301242,5.053778
3,3.217117,5.548506
4,3.255706,4.662450


>From the result above, we can easily find out that train_RMSE is lower when M = 2, but CV_RMSE gets higher simultaneously.  
We may say it's the consequence of overfitting becuase the model is too complex.  

### b. Please analyze the **weights of polynomial models for $\rm M = 1$** and select the most contributive attribute which has the lowest RMS error on the Training Dataset.  

In [16]:
M_1[1]

array([[-1.95326765e+01],
       [ 2.28682316e-02],
       [ 1.89471675e+01],
       [ 2.29667125e+01],
       [-2.80782751e+01],
       [ 1.45658668e+00],
       [ 1.91586996e+00],
       [-1.76386087e+00],
       [ 3.14089560e-02],
       [ 4.15924627e-01],
       [-9.32115418e-01],
       [ 6.21448417e-02],
       [ 5.29546268e-01],
       [-1.30950521e+01],
       [ 4.34440115e-02],
       [-3.87311907e-02],
       [ 2.76724687e+00],
       [-4.45916499e+00]])

> The list above is the $w$ of the model which provides the lowest CV_RMSE when $\rm M = 1$.  

## 2. Maximum Likelihood Approach

### a. **Choose some of air quality measurement** in dataset X.csv and design your model. You can choose any basis functions you like and implemented the feature vector.  

> Here I **selected 10 air quality measurements** as independent variables, and conducted a cross-validation process to make sure that the model won't be over-fitting.  

In [17]:
M_G_10 = cross_val(data_x, data_t, 5, 10, 0, get_gaussphi)

by RMSE, we choose fold 0 for the training.


In [18]:
M_G_10[0]

,train_RMSE,CV_RMSE
0,8.620325,8.105435
1,8.444232,8.665498
2,8.539940,8.473902
3,8.450909,8.761456
4,8.366492,9.104294


### b. Apply N-fold cross-validation in your training stage to select at least one hyperparameter (order, parameter number, ...) for model and do some discussion (underfitting, overfitting).  


In [19]:
w_SP = search_hyper(data_x, data_t, 7, 18, 1, get_gaussphi)

iter: 0
by RMSE, we choose fold 0 for the training.
iter: 1
by RMSE, we choose fold 5 for the training.
iter: 2
by RMSE, we choose fold 2 for the training.
iter: 3
by RMSE, we choose fold 5 for the training.
iter: 4
by RMSE, we choose fold 0 for the training.
iter: 5
by RMSE, we choose fold 1 for the training.
iter: 6
by RMSE, we choose fold 5 for the training.
iter: 7
by RMSE, we choose fold 5 for the training.
iter: 8
by RMSE, we choose fold 6 for the training.
iter: 9
by RMSE, we choose fold 5 for the training.
iter: 10
by RMSE, we choose fold 6 for the training.
iter: 11
by RMSE, we choose fold 4 for the training.
iter: 12
by RMSE, we choose fold 3 for the training.
iter: 13
by RMSE, we choose fold 3 for the training.
iter: 14
by RMSE, we choose fold 4 for the training.
iter: 15
by RMSE, we choose fold 4 for the training.
iter: 16
by RMSE, we choose fold 1 for the training.
iter: 17
by RMSE, we choose fold 1 for the training.

From iteration 13, we got the lowest CV_RMSE: 7.3783448

> **w_opt** is shown in the chunk below.  

In [20]:
w_SP

[            0
 0   24.623731
 1    6.364524
 2   -3.827172
 3   -5.234192
 4   -1.425304
 5   -9.238251
 6   -2.678211
 7    3.834383
 8    0.706080
 9  -13.001768
 10  10.864498
 11  -0.356424
 12  -4.618514
 13   8.005912, 13]

## 3. Maximum a posteriori approach

### a. Use maximum a posteriori approach method and repeat **2.(a)** and **2.(b)**. You could choose Gaussian distribution as a prior.  

> I'll choose **Gaussian basis function** here, and try to calculate the posterior distribution.  
> 
> Add a Gaussian noise to the model:  
> $$\epsilon \sim N(0,\ \beta)$$  
> We may renew our parameters by the functions below:   
> 
> $$p(w|t) = N(w|m_N,\ S_N)\ \rm{,where\ }$$  
> $$S_N^{-1} = S_0^{-1} + \beta \Phi^T \Phi$$  
> $$m_N = S_N (S_0^{-1}m_0 + \beta \Phi^T y)$$

In [21]:
M = w_SP[1]
m0 = np.zeros(M+1).reshape(-1,1)
s0 = 2*np.eye(M+1)
beta = 0.2


In [22]:
mn = batch_learn(data_x, data_t, 100, s0, m0, M)[1]

In [23]:
M_G_ML = cross_val(data_x, data_t, 7, 5, 0, get_gaussphi)

by RMSE, we choose fold 1 for the training.


### b. Compare the result between maximum likelihood approach and maximum a posteriori approach.  

> The results below, I'll show the RMSE of $w$ from MAP first and then from Maximize Likelihood.  

In [24]:
((np.ones(data_t.shape[0]) @ ((get_gaussphi(data_x, M) @ mn - data_t.values)**2))/data_t.shape[0])**0.5

0    8.389532
dtype: float64

In [25]:
M_G_ML[0]

,train_RMSE,CV_RMSE
0,9.486842,8.925995
1,9.524986,8.611880
2,9.358976,9.679364
3,9.332668,9.828988
4,9.347740,9.689268
5,9.346682,9.744369
6,9.335558,9.726591
